In [1]:
from phylonetwork import PhyloNetwork, PhyloTree, LGTPhyloNetwork
import phylonetwork
import random
from numpy import matrix
from collections import Counter, deque
import os,glob
import re
import itertools
import networkx as nx
import copy

In [2]:
# path where I have the level-k generators from Gambette
filename = './Lev5Gen.txt'

In [3]:
# read and save the generators in lists for each level
g=[]
conti=True 
i=0
#f.close()
f=open(filename,"r")
reads=f.readlines()
f.close()
while conti==True:
    if reads[i].find("Beginning")!=-1:
        g.append('CHANGE LEVEL')
    if reads[i].find("{")!=-1:
        g.append('NEW')
        j=i+1
        while reads[j].find("}")==-1:
            g.append(reads[j])
            j+=1
        i=j
    if reads[i].find("End of level-5")!=-1:
        conti=False 
    i+=1

def obtain_arc(s):
    nodes=re.findall(r'\d+', s)
    return(int(nodes[0]),int(nodes[1]))

GENS = {}
for i in range(1,6):
    GENS[i] = []
edges=[]
level=0
i=0
while i<len(g):
    if g[i].find("CHANGE LEVEL")!=-1:
        level+=1
    elif g[i].find("NEW")!=-1:
        graph = nx.MultiDiGraph()
        #graph=PhyloNetwork()
    else:
        edges.append(obtain_arc(g[i]))
        if (i+1==len(g)) or (g[i+1].find("NEW")!=-1) or (g[i+1].find("CHANGE LEVEL")!=-1):
            graph.add_edges_from(edges) 
            edges=[]
            GENS[level].append(graph)
            
    i+=1

In [4]:
[len(GENS[l]) for l in GENS]

[1, 4, 65, 1993, 91454]

In [5]:
def taggings1(generator):
    """
    First step: for each level-k generator, and for each of its hybrid nodes, tag
    exactly one of the incoming arcs with 'SEC', meaning that they will be secondary arcs.
    """
    hybrid_nodes = [u for u in generator.nodes() if generator.in_degree(u) == 2]
    ll = []
    for h in hybrid_nodes:
        ll.append(generator.in_edges(h, keys = True))
    for choice in itertools.product(*ll):
        new_graph = generator.copy()
        for edge in choice:
            new_graph[edge[0]][edge[1]][edge[2]]['tag'] = 'SEC'
        yield new_graph
    return

In [6]:
def taggings2(generator):
    """
    Second step: for each of the networks generated in the first step, find all nodes having two
    outgoing arcs tagged with 'SEC' and change exactly one of them into 'MIX', meaning that
    this arc will be split into a principal arc followed by a secondary arc.
    """
    for generator1 in taggings1(generator):
        hybrid_nodes = [u for u in generator.nodes() if generator.in_degree(u) == 2]
        for h in hybrid_nodes:
            for edge in generator1.out_edges(h, keys=True, data=True):
                if edge[-1].get('tag') == 'SEC':
                    generator1[edge[0]][edge[1]][edge[2]]['tag'] = 'MIX'
        tree_nodes = [u for u in generator1.nodes() if generator1.out_degree(u) == 2]
        ll = []
        for t in tree_nodes:
            outgoing = generator1.out_edges(t, keys=True, data=True)
            if outgoing[0][-1].get('tag') == 'SEC' and outgoing[1][-1].get('tag') == 'SEC':
                ll.append(outgoing)
        for choice in itertools.product(*ll):
            new_graph = generator1.copy()
            for edge in choice:
                new_graph[edge[0]][edge[1]][edge[2]]['tag'] = 'MIX'
            yield new_graph

In [7]:
em = nx.isomorphism.categorical_multiedge_match('tag','')
def taggings_up_to_isomorphism(generator):
    """
    Third step: Find all possible taggings for a generator and filter out repetitions
    due to isomorphism of edge-tagged graphs
    """
    all_taggings = list(taggings2(generator))
    uniques = []
    for g in all_taggings:
        for h in uniques:
            if nx.is_isomorphic(g,h, edge_match = em):
                break
        else:
            uniques.append(g)
    return uniques

In [8]:
# Compute all LGT-generators
LGTGENS = {}
for i in range(1,6):
    LGTGENS[i] = []
    for generator in GENS[i]:
        LGTGENS[i].extend(taggings_up_to_isomorphism(generator))
    print(f'Level {i}: {len(LGTGENS[i])} generators')

Level 1: 1 generators
Level 2: 14 generators
Level 3: 546 generators
Level 4: 39257 generators


KeyboardInterrupt: 